<a href="https://colab.research.google.com/github/shadow347/dw_matrix_road_sign/blob/master/day_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [18]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score
from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign"

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle("data/train.p")
test = pd.read_pickle("data/test.p")

X_train, y_train = train["features"], train["labels"]
X_test, y_test = test["features"], test["labels"]

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit ={}):
  model.compile(loss = "categorical_crossentropy", optimizer = "Adam", metrics = ["accuracy"])
  
  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq = 1)

  model.fit(
      X_train, 
      y_train,
      batch_size = params_fit.get("batch_size", 128),
      epochs = params_fit.get("epochs", 5),
      verbose = params_fit.get("verbose", 1),
      validation_data = params_fit.get("validation_data", (X_train, y_train)),
      callbacks = [tensorboard_callback]
      )
  
  return model


  
def predict(model_trained, X_test, y_test, scoring = accuracy_score):
 
  y_test_norm = np.argmax(y_test, axis = 1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis = 1)

  return scoring(y_test_norm, y_pred)


In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
     Conv2D(filters = 32, kernel_size=(3, 3), activation= "relu", input_shape = input_shape),
     Conv2D(filters = 32, kernel_size=(3, 3), activation= "relu", padding="same"),
     MaxPool2D(),
     Dropout(0.3),

     Conv2D(filters = 64, kernel_size=(3, 3), activation= "relu", padding="same"),
     Conv2D(filters = 64, kernel_size=(3, 3), activation= "relu"),
     MaxPool2D(),
     Dropout(0.3),

     Conv2D(filters = 64, kernel_size=(3, 3), activation= "relu", padding="same"),
     Conv2D(filters = 64, kernel_size=(3, 3), activation= "relu"),
     MaxPool2D(),
     Dropout(0.3),

     Flatten(),

     Dense(1024, activation="relu"),
     Dropout(0.3),

     Dense(1024, activation="relu"),
     Dropout(0.3),

     Dense(num_classes, activation="softmax")             
                    
])



In [8]:
model = get_cnn_v5(input_shape, num_classes)

model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 4s 14ms/step - loss: 2.6482 - accuracy: 0.2874 - val_loss: 1.1854 - val_accuracy: 0.6572
Epoch 2/5
272/272 [==============================] - 4s 13ms/step - loss: 1.0088 - accuracy: 0.6786 - val_loss: 0.3322 - val_accuracy: 0.8894
Epoch 3/5
272/272 [==============================] - 4s 13ms/step - loss: 0.5620 - accuracy: 0.8215 - val_loss: 0.1724 - val_accuracy: 0.9437
Epoch 4/5
272/272 [==============================] - 4s 13ms/step - loss: 0.3864 - accuracy: 0.8782 - val_loss: 0.1073 - val_accuracy: 0.9705
Epoch 5/5
272/272 [==============================] - 4s 13ms/step - loss: 0.2754 - accuracy: 0.9123 - val_loss: 0.0798 - val_accuracy: 0.9748


0.9541950113378684

In [13]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 0s 2ms/step - loss: 0.2391 - accuracy: 0.9542


0.9541950225830078

In [0]:
def get_model(params):
  return Sequential([
     Conv2D(filters = 32, kernel_size=(3, 3), activation= "relu", input_shape = input_shape),
     Conv2D(filters = 32, kernel_size=(3, 3), activation= "relu", padding="same"),
     MaxPool2D(),
     Dropout(params["dropout_cnn_block_one"]),

     Conv2D(filters = 64, kernel_size=(3, 3), activation= "relu", padding="same"),
     Conv2D(filters = 64, kernel_size=(3, 3), activation= "relu"),
     MaxPool2D(),
     Dropout(params["dropout_cnn_block_two"]),

     Conv2D(filters = 128, kernel_size=(3, 3), activation= "relu", padding="same"),
     Conv2D(filters = 128, kernel_size=(3, 3), activation= "relu"),
     MaxPool2D(),
     Dropout(params["dropout_cnn_block_three"]),

     Flatten(),

     Dense(1024, activation="relu"),
     Dropout(params["dropout_dense_block_one"]),

     Dense(1024, activation="relu"),
     Dropout(params["dropout_dense_block_two"]),

     Dense(num_classes, activation="softmax")             
                    
])


In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss = "categorical_crossentropy", optimizer = "Adam", metrics = ["accuracy"])
  
  

  model.fit(
      X_train, 
      y_train,
      batch_size = int(params.get("batch_size", 128)),
      epochs = 5,
      verbose = 0
     
      )
  
  score = model.evaluate(X_test, y_test, verbose = 0)
  accuracy = score[1]
  print(params, "accuracy = {}".format(accuracy))
  return {"loss": -accuracy, "status": STATUS_OK, "model": model}


In [37]:
space = {
    "batch_size": hp.quniform("batch_size", 50, 200, 20),
    "dropout_cnn_block_one": hp.uniform("dropout_cnn_block_one", 0, 1),
    "dropout_cnn_block_two": hp.uniform("dropout_cnn_block_two", 0, 1),
    "dropout_cnn_block_three": hp.uniform("dropout_cnn_block_three", 0, 1),

    "dropout_dense_block_one": hp.uniform("dropout_dense_block_one", 0, 1),
    "dropout_dense_block_two": hp.uniform("dropout_dense_block_two", 0, 1)
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()


)

{'batch_size': 100.0, 'dropout_cnn_block_one': 0.9646460903915869, 'dropout_cnn_block_three': 0.1281751043571836, 'dropout_cnn_block_two': 0.959043931455636, 'dropout_dense_block_one': 0.7716206071058772, 'dropout_dense_block_two': 0.06775365170966441}
accuracy = 0.05646258592605591
{'batch_size': 60.0, 'dropout_cnn_block_one': 0.40351362105354593, 'dropout_cnn_block_three': 0.5018685473281848, 'dropout_cnn_block_two': 0.6949363548341687, 'dropout_dense_block_one': 0.9250767308836667, 'dropout_dense_block_two': 0.70097962236714}
accuracy = 0.28730159997940063
{'batch_size': 160.0, 'dropout_cnn_block_one': 0.9689694841404829, 'dropout_cnn_block_three': 0.4419831104104135, 'dropout_cnn_block_two': 0.9426614434459798, 'dropout_dense_block_one': 0.32060687883604855, 'dropout_dense_block_two': 0.49560027426991693}
accuracy = 0.048072561621665955
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.4381429207423976, 'dropout_cnn_block_three': 0.5137093560255959, 'dropout_cnn_block_two': 0.002866